**Link da base de dados** - https://www.kaggle.com/datasets/thedevastator/books-sales-and-ratings

In [ ]:
# Importação da biblioteca pandas
import pandas as pd

In [ ]:
# Instalação dos requisitos para o PySpark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
# Configurar as variáveis de ambiente
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
# Torna o pyspark "importável"
import findspark
findspark.init()

In [ ]:
# iniciar uma sessão local
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Introducao").getOrCreate()

In [ ]:
# Verifica o SparkContext
print(spark)

# Exibe a Spark version
print(spark.version)

3.1.1


**INSTRUÇÕES** - 4 - Elabore Análises com filtros e agrupamentos aprendidos em PySpark na aula de ontem; 5 - Cada filtro ou agrupamento que você realizar deve estar acompanhado (no bloco de código do respectivo colab) de um comentário sobre qual pergunta essas dados filtrados ou agrupados tentam responder sobre essa base; 6 - Cada filtro ou agrupamento deverá ser salvo em um objeto PySpark e depois convertido em um objeto Pandas, ao longo do dia você deverá realizar no mínimo 20 tabelas resultantes dessas análises; 7 - Após isso, as tabelas pandas deverão ser enviadas diretamente do colab para arquivos csv.

In [ ]:
# Definir quais funcões sql pyspark vamos importar
from pyspark.sql.functions import round, col, collect_list, when, count, sum, max, avg
from pyspark.sql import functions as F
from IPython.display import display

#Carregando um arquivo CSV em um DataFrame
df = spark.read.csv("/content/drive/MyDrive/Colab Notebooks/Datasets/Books_Data_Clean.csv", header=True, inferSchema=True)
df = df.dropna()
# Mostrar as primeiras linhas do DataFrame
#df.show()

In [ ]:
# 1 - Quantidade de livros publicados por cada ano

df_livros_ano = df.groupBy("Publishing Year").agg(count("Book Name").alias("Total de livros publicados por ano"))
df_livros_ano.orderBy(col("Publishing Year").desc()).show() #ordenar do ano mais recente
df_livros_ano_pd = df_livros_ano.toPandas() #Converte para pandas
df_livros_ano_pd.to_csv("livros_publicados_ano.csv", index=False) #Salva em csv

+---------------+----------------------------------+
|Publishing Year|Total de livros publicados por ano|
+---------------+----------------------------------+
|         2016.0|                                10|
|         2015.0|                                25|
|         2014.0|                                33|
|         2013.0|                                31|
|         2012.0|                                59|
|         2011.0|                                54|
|         2010.0|                                41|
|         2009.0|                                45|
|         2008.0|                                37|
|         2007.0|                                32|
|         2006.0|                                40|
|         2005.0|                                34|
|         2004.0|                                29|
|         2003.0|                                38|
|         2002.0|                                17|
|         2001.0|                             

In [ ]:
# 2 - Quantidade de livros publicados por editora

df_livros_editora = df.groupBy("Publisher ").agg(count("Book Name").alias("Total de livros publicados por editora"))
df_livros_editora.orderBy("Total de livros publicados por editora").show()
df_livros_editora_pd = df_livros_editora.toPandas() #Converte para pandas
df_livros_editora_pd.to_csv("livros_publicados_editora.csv", index=False) #Salva em csv

+--------------------+--------------------------------------+
|          Publisher |Total de livros publicados por editora|
+--------------------+--------------------------------------+
|HarperCollins Pub...|                                     4|
|HarperCollins Chr...|                                     4|
|           Macmillan|                                    40|
|Simon and Schuste...|                                    55|
| Hachette Book Group|                                    56|
|HarperCollins Pub...|                                    65|
|Penguin Group (US...|                                    97|
|    Random House LLC|                                   111|
|Amazon Digital Se...|                                   566|
+--------------------+--------------------------------------+



In [ ]:
# 3 - Quantidade de livros publicados por gênero

df_livros_genero = df.groupBy("genre").agg(count("Book Name").alias("Total de livros publicados por gênero"))
df_livros_genero.orderBy("Total de livros publicados por gênero").show()
df_livros_genero_pd = df_livros_genero.toPandas() #Converte para pandas
df_livros_genero_pd.to_csv("livros_publicados_genero.csv", index=False) #Salva em csv

+-------------+-------------------------------------+
|        genre|Total de livros publicados por gênero|
+-------------+-------------------------------------+
|     children|                                   15|
|      fiction|                                   55|
|   nonfiction|                                  156|
|genre fiction|                                  772|
+-------------+-------------------------------------+



In [ ]:
#4 - Autor com o maior número de livros publicados

df_autor_livro = df.groupBy("Author").agg(count("Book Name").alias("Total de livros publicados por autor"))
autor_mais_publicado = df_autor_livro.orderBy(col("Total de livros publicados por autor").desc()).first()
#df_autor_livro.orderBy("Total de livros publicados por autor").show()
print(f"O autor com o maior número de livros publicados é {autor_mais_publicado['Author']} com {autor_mais_publicado['Total de livros publicados por autor']} títulos.")
df_autor_livro_pd = df_autor_livro.toPandas() #Converte para pandas
df_autor_livro_pd.to_csv("livros_publicados_autor.csv", index=False)

O autor com o maior número de livros publicados é Stephen King com 27 títulos.


In [ ]:
#5 - Autor com menor número e autor com o maior número de livros vendidos

df_autor_vendas = df.groupBy("Author").agg(sum("units sold").alias("Total de vendas por autor"))
autor_maior_vendas = df_autor_vendas.orderBy(col("Total de vendas por autor").desc()).first()
autor_menor_vendas = df_autor_vendas.orderBy(col("Total de vendas por autor").asc()).first()
print(f"O autor com o maior número de livros vendidos é {autor_maior_vendas['Author']} com {autor_maior_vendas['Total de vendas por autor']} livros vendidos.")
print(f"O autor com o menor número de livros vendidos é {autor_menor_vendas['Author']} com {autor_menor_vendas['Total de vendas por autor']} livros vendidos.")
df_autor_vendas_pd = df_autor_vendas.toPandas() #Converte para pandas
df_autor_vendas_pd.to_csv("livros_vendas_autor.csv", index=False)

O autor com o maior número de livros vendidos é Stephen King com 270824 livros vendidos.
O autor com o menor número de livros vendidos é Bryce Courtenay com 106 livros vendidos.


In [ ]:
# 6 - Total de vendas por editora

df_editora_vendas = df.groupBy("Publisher ").agg(sum("units sold").alias("Total de vendas por editora"))
df_editora_vendas.orderBy("Total de vendas por editora").show()
df_editora_vendas_pd = df_editora_vendas.toPandas() #Converte para pandas
df_editora_vendas_pd.to_csv("livros_vendas_editora.csv", index=False)

+--------------------+---------------------------+
|          Publisher |Total de vendas por editora|
+--------------------+---------------------------+
|HarperCollins Chr...|                      41527|
|HarperCollins Pub...|                      44358|
|           Macmillan|                     303177|
| Hachette Book Group|                     457760|
|Simon and Schuste...|                     501573|
|HarperCollins Pub...|                     540643|
|Penguin Group (US...|                     841540|
|    Random House LLC|                    1255080|
|Amazon Digital Se...|                    5797050|
+--------------------+---------------------------+



In [ ]:
#7 - Total de livros vendidos por gênero

df_vendas_genero = df.groupBy("genre").agg(sum("units sold").alias("Total de livros vendidos por gênero"))
df_vendas_genero.orderBy("Total de livros vendidos por gênero").show()
df_vendas_genero_pd = df_vendas_genero.toPandas() #Converte para pandas
df_vendas_genero_pd.to_csv("livros_vendas_genero.csv", index=False) #Salva em csv

+-------------+-----------------------------------+
|        genre|Total de livros vendidos por gênero|
+-------------+-----------------------------------+
|     children|                             298118|
|      fiction|                             450025|
|   nonfiction|                            1558281|
|genre fiction|                            7476284|
+-------------+-----------------------------------+



In [ ]:
#8 - Livros com maior avaliação (ordenando do maior para o menor)

df_livros_avaliacao = df.groupBy("Book Name").agg(max("Book_average_rating").alias("Avaliação do livro"))
df_livros_avaliacao = df_livros_avaliacao.orderBy(col("Avaliação do livro").desc())
df_livros_avaliacao.show()
df_livros_avaliacao_pd = df_livros_avaliacao.toPandas() #Converte para pandas
df_livros_avaliacao_pd.to_csv("livros_avaliacao.csv", index=False) #Salva em csv

+--------------------+------------------+
|           Book Name|Avaliação do livro|
+--------------------+------------------+
|   Words of Radiance|              4.77|
|A Court of Mist a...|              4.72|
|The Essential Cal...|              4.65|
|    The Way of Kings|              4.64|
|   Calvin and Hobbes|              4.61|
|    Queen of Shadows|               4.6|
|The Hobbit and Th...|              4.59|
|A Storm of Swords...|              4.56|
|  The House of Hades|              4.54|
|   The Complete Maus|              4.53|
|        Heir of Fire|              4.53|
|   A Memory of Light|               4.5|
|The Complete Sher...|               4.5|
|   Crown of Midnight|              4.49|
|é‹¼ã®éŒ¬é‡‘è¡“å¸« 1|              4.49|
|              Winter|              4.49|
|City of Heavenly ...|              4.48|
|       Lonesome Dove|              4.47|
|         Night Watch|              4.47|
|               Cress|              4.46|
+--------------------+------------

In [ ]:
#9 - Avaliação média dos livros publicados por editora

df_editora_avaliacao = df.groupBy("Publisher ").agg(avg("Book_average_rating").alias("Avaliação média dos livros publicados"))
df_editora_avaliacao = df_editora_avaliacao.orderBy(col("Avaliação média dos livros publicados").desc())
df_editora_avaliacao= df_editora_avaliacao.withColumn("Avaliação média dos livros publicados", round("Avaliação média dos livros publicados",2))
df_editora_avaliacao.show()
df_editora_avaliacao_pd = df_editora_avaliacao.toPandas() #Converte para pandas
df_editora_avaliacao_pd.to_csv("livros_avaliacao_editora.csv", index=False) #Salva em csv


+--------------------+-------------------------------------+
|          Publisher |Avaliação média dos livros publicados|
+--------------------+-------------------------------------+
|HarperCollins Chr...|                                 4.17|
|HarperCollins Pub...|                                 4.13|
|Penguin Group (US...|                                 4.04|
|Simon and Schuste...|                                 4.04|
|Amazon Digital Se...|                                  4.0|
|           Macmillan|                                  4.0|
|HarperCollins Pub...|                                  4.0|
|    Random House LLC|                                 3.98|
| Hachette Book Group|                                 3.94|
+--------------------+-------------------------------------+



In [ ]:
#10 - Quantidade de autores de acordo com a classificação - Novice, Famous, Intermidiate

df_classificacao_autores = df.groupBy("Author_Rating").agg(count("Author").alias("Total de autores"))
df_classificacao_autores = df_classificacao_autores.orderBy(col("Total de autores").desc())
df_classificacao_autores.show()
df_classificacao_autores_pd = df_classificacao_autores.toPandas() #Converte para pandas
df_classificacao_autores_pd.to_csv("classificacao_autores.csv", index=False) #Salva em csv

+-------------+----------------+
|Author_Rating|Total de autores|
+-------------+----------------+
| Intermediate|             589|
|    Excellent|             333|
|       Famous|              47|
|       Novice|              29|
+-------------+----------------+



In [ ]:
#11 - Livros de acordo com sale price

cortes = [0, 5, 10, 15, 20]
rotulos = ['0-4.99', '5-9.99', '10-14.99', '15-19.99']

df = df.withColumn(
    'Faixa_Preco',
    when(col('sale price').between(cortes[0], cortes[1] - 1), rotulos[0])
    .when(col('sale price').between(cortes[1], cortes[2] - 1), rotulos[1])
    .when(col('sale price').between(cortes[2], cortes[3] - 1), rotulos[2])
    .when(col('sale price').between(cortes[3], cortes[4] - 1), rotulos[3])
    .otherwise(None)  # Para valores fora do intervalo de preço
)

df_preco_livros = df.groupBy("Faixa_Preco").agg(count("Book Name").alias("Total de livros"))
df_preco_livros.orderBy(col("Faixa_Preco")).show()
df_preco_livros_pd = df_preco_livros.toPandas() #Converte para pandas
df_preco_livros_pd.to_csv("classificacao_livros_preco.csv", index=False) #Salva em csv

+-----------+---------------+
|Faixa_Preco|Total de livros|
+-----------+---------------+
|       null|            132|
|     0-4.99|            550|
|   10-14.99|             87|
|   15-19.99|              3|
|     5-9.99|            226|
+-----------+---------------+



In [ ]:
#12 - Autor com o maior número de avaliações

df_autor_mais_avaliado = df.groupBy("Author").agg(sum("Book_ratings_count").alias("Total de avaliações"))
#df_autor_mais_avaliado.show()
autor_mais_avaliado = df_autor_mais_avaliado.orderBy((col("Total de avaliações").desc())).first()
print(f"O autor com o maior número de avaliações é {autor_mais_avaliado['Author']} com {autor_mais_avaliado['Total de avaliações']} avaliações.")
df_autor_mais_avaliado_pd = df_autor_mais_avaliado.toPandas() #Converte para pandas
df_autor_mais_avaliado_pd.to_csv("avaliacoes_autores.csv", index=False) #Salva em csv

O autor com o maior número de avaliações é Stephen King com 2588612 avaliações.


In [ ]:
#13 - Total de vendas por gênero

df_vendas_genero = df.groupBy("genre").agg(sum("gross sales").alias("Total de vendas"))
df_vendas_genero = df_vendas_genero.orderBy(col("Total de vendas").desc())
df_vendas_genero= df_vendas_genero.withColumn("Total de vendas", round("Total de vendas",2))
df_vendas_genero.show()
df_vendas_genero_pd = df_vendas_genero.toPandas() #Converte para pandas
df_vendas_genero_pd.to_csv("vendas_genero.csv", index=False) #Salva em csv

+-------------+---------------+
|        genre|Total de vendas|
+-------------+---------------+
|genre fiction|     1535794.78|
|   nonfiction|      215894.42|
|      fiction|      115723.56|
|     children|       13902.22|
+-------------+---------------+



In [ ]:
#14 - Média de preço de venda por gênero

df_media_preco_genero = df.groupBy("genre").agg(avg("sale price").alias("Preço média de vendas"))
df_media_preco_genero = df_media_preco_genero.orderBy(col("Preço média de vendas").desc())
df_media_preco_genero= df_media_preco_genero.withColumn("Preço média de vendas", round("Preço média de vendas",2))
df_media_preco_genero.show()
df_media_preco_genero_pd = df_media_preco_genero.toPandas() #Converte para pandas
df_media_preco_genero_pd.to_csv("media_preco_genero.csv", index=False) #Salva em csv

+-------------+---------------------+
|        genre|Preço média de vendas|
+-------------+---------------------+
|     children|                 6.13|
|   nonfiction|                 5.53|
|      fiction|                 5.12|
|genre fiction|                 4.66|
+-------------+---------------------+



In [ ]:
#15 - Autor Novice/Novato com maior número de vendas

df_autor_novatos = df.filter(col("Author_Rating") == "Novice")
#df_autor_novatos.show()
df_autor_novatos_vendas = df_autor_novatos.groupBy("Author").agg(sum("units sold").alias("Total de vendas"))
autor_novato_maior_venda = df_autor_novatos_vendas.orderBy((col("Total de vendas").desc())).first()
print(f"O autor novato com o maior número de vendas é {autor_novato_maior_venda['Author']} com {autor_novato_maior_venda['Total de vendas']} livros vendidos.")
df_autor_novatos_vendas_pd = df_autor_novatos_vendas.toPandas() #Converte para pandas
df_autor_novatos_vendas_pd.to_csv("autor_novatos_vendas.csv", index=False) #Salva em csv


O autor novato com o maior número de vendas é Jane Green com 61128 livros vendidos.


In [ ]:
#16 - Média de avaliação dos livros de Autores Famous

df_autores_famous = df.filter(col("Author_Rating") == "Famous")
#df_autores_famous.show()
df_famous_media = df_autores_famous.groupBy("Author").agg(avg("Book_average_rating").alias("Média de avaliação dos livros"))
df_famous_media = df_famous_media.withColumn("Média de avaliação dos livros", round("Média de avaliação dos livros",2))
df_famous_media = df_famous_media.orderBy((col("Média de avaliação dos livros").desc()))
df_famous_media.show()
df_famous_media_pd = df_famous_media.toPandas()
df_famous_media_pd.to_csv("autores_famosos_avaliacao.csv", index=False)

+--------------------+-----------------------------+
|              Author|Média de avaliação dos livros|
+--------------------+-----------------------------+
|      Bill Watterson|                         4.65|
|   Brandon Sanderson|                         4.62|
|Bill Watterson, G...|                         4.61|
|       Sarah J. Maas|                         4.59|
|      J.R.R. Tolkien|                         4.59|
|  George R.R. Martin|                         4.56|
|  Arthur Conan Doyle|                          4.5|
|Hiromu Arakawa, A...|                         4.49|
|       Marissa Meyer|                         4.48|
|        Rick Riordan|                         4.48|
|     Cassandra Clare|                         4.48|
|      Art Spiegelman|                         4.47|
|      Larry McMurtry|                         4.47|
|     Terry Pratchett|                         4.47|
|Robert Jordan, Br...|                         4.46|
|        Pierce Brown|                        

In [ ]:
#17 - Editora com o maior número de autores Famous

df_autores_famous = df.filter(col("Author_Rating") == "Famous")
#df_autores_famous.show()
df_famous_editora = df_autores_famous.groupBy("Publisher ").agg(count("Author").alias("Total de autores famosos"))
df_famous_editora = df_famous_editora.orderBy((col("Total de autores famosos").desc()))
df_famous_editora.show()
df_famous_editora_pd = df_famous_editora.toPandas()
df_famous_editora_pd.to_csv("autores_famosos_editora.csv", index=False)

+--------------------+------------------------+
|          Publisher |Total de autores famosos|
+--------------------+------------------------+
|Amazon Digital Se...|                      32|
|Penguin Group (US...|                       4|
|Simon and Schuste...|                       4|
|           Macmillan|                       3|
| Hachette Book Group|                       2|
|    Random House LLC|                       1|
|HarperCollins Pub...|                       1|
+--------------------+------------------------+



In [ ]:
#18 - Editora com o maior número de autores Novice

df_autor_novatos = df.filter(col("Author_Rating") == "Novice")
#df_autor_novatos.show()
df_autor_novatos_editora = df_autor_novatos.groupBy("Publisher ").agg(count("Author").alias("Total de autores novatos"))
df_autor_novatos_editora = df_autor_novatos_editora.orderBy((col("Total de autores novatos").desc()))
df_autor_novatos_editora.show()
df_autor_novatos_editora_pd = df_autor_novatos_editora.toPandas() #Converte para pandas
df_autor_novatos_editora_pd.to_csv("autor_novatos_editora.csv", index=False) #Salva em csv

+--------------------+------------------------+
|          Publisher |Total de autores novatos|
+--------------------+------------------------+
|Amazon Digital Se...|                      15|
|Penguin Group (US...|                       4|
|    Random House LLC|                       4|
|HarperCollins Pub...|                       3|
| Hachette Book Group|                       1|
|           Macmillan|                       1|
|Simon and Schuste...|                       1|
+--------------------+------------------------+



In [ ]:
#19 - Média de preço de livros por editora

df_media_preco_editora = df.groupBy("Publisher ").agg(avg("sale price").alias("Média de preço de venda"))
df_media_preco_editora = df_media_preco_editora.withColumn("Média de preço de venda", round("Média de preço de venda",2))
df_media_preco_editora = df_media_preco_editora.orderBy((col("Média de preço de venda").desc()))
df_media_preco_editora.show()
df_media_preco_editora_pd = df_media_preco_editora.toPandas()
df_media_preco_editora_pd.to_csv("media_preco_venda_editora.csv", index=False)

+--------------------+-----------------------+
|          Publisher |Média de preço de venda|
+--------------------+-----------------------+
|Penguin Group (US...|                   8.61|
|    Random House LLC|                   7.26|
|HarperCollins Pub...|                   6.51|
| Hachette Book Group|                   6.45|
|Simon and Schuste...|                   5.94|
|HarperCollins Pub...|                    5.7|
|           Macmillan|                   5.36|
|HarperCollins Chr...|                   5.24|
|Amazon Digital Se...|                   3.21|
+--------------------+-----------------------+



In [ ]:
#20 - Média de gross sales por editora

df_media_sales_editora = df.groupBy("Publisher ").agg(avg("gross sales").alias("Média de vendas brutas"))
df_media_sales_editora = df_media_sales_editora.withColumn("Média de vendas brutas", round("Média de vendas brutas",2))
df_media_sales_editora = df_media_sales_editora.orderBy((col("Média de vendas brutas").desc()))
df_media_sales_editora.show()
df_media_sales_editora_pd = df_media_sales_editora.toPandas()
df_media_sales_editora_pd.to_csv("media_preco_vendabrutas_editora.csv", index=False)

+--------------------+----------------------+
|          Publisher |Média de vendas brutas|
+--------------------+----------------------+
| Hachette Book Group|                3828.7|
|Penguin Group (US...|               3388.35|
|HarperCollins Pub...|               3212.82|
|    Random House LLC|               2573.22|
|Simon and Schuste...|               1411.04|
|           Macmillan|                1328.2|
|Amazon Digital Se...|               1245.17|
|HarperCollins Pub...|                1179.5|
|HarperCollins Chr...|                889.86|
+--------------------+----------------------+



In [ ]:
#21 - Média de gross sales por autores Famous

df_autores_famous = df.filter(col("Author_Rating") == "Famous")
#df_autores_famous.show()
df_famous_editora_vendas = df_autores_famous.groupBy("Author").agg(avg("gross sales").alias("Média de vendas brutas"))
df_famous_editora_vendas = df_famous_editora_vendas.withColumn("Média de vendas brutas", round("Média de vendas brutas",2))
df_famous_editora_vendas = df_famous_editora_vendas.orderBy((col("Média de vendas brutas").desc()))
df_famous_editora_vendas.show()
df_famous_editora_vendas_pd = df_famous_editora_vendas.toPandas()
df_famous_editora_vendas_pd.to_csv("autores_famosos_editora_vendasbrutas.csv", index=False)

+--------------------+----------------------+
|              Author|Média de vendas brutas|
+--------------------+----------------------+
|Tsugumi Ohba, Tak...|               6687.24|
|     Cassandra Clare|               6676.62|
|       Richelle Mead|               3431.34|
|  Arthur Conan Doyle|               3180.02|
|   Brandon Sanderson|               2915.39|
|Bill Watterson, G...|               1886.22|
|      Larry McMurtry|               1687.77|
|      Diana Gabaldon|               1568.87|
|           J.R. Ward|                978.04|
|     Terry Pratchett|                910.86|
|       Marissa Meyer|                 776.0|
|      Colleen Hoover|                747.93|
|         Jim Butcher|                669.81|
|        Stephen King|                612.25|
|       Sarah J. Maas|                524.68|
|    Ta-Nehisi Coates|                523.25|
|          Alex Haley|                460.35|
|EiichirÅ Oda, An...|                458.85|
|Hiro Mashima, Wil...|            

In [ ]:
#22 - Média de receita do editor por editora

df_media_receita = df.groupBy("Publisher ").agg(avg("publisher revenue").alias("Média de receita para editor"))
df_media_receita = df_media_receita.withColumn("Média de receita para editor", round("Média de receita para editor",2))
df_media_receita = df_media_receita.orderBy((col("Média de receita para editor").desc()))
df_media_receita.show()
df_media_receita_pd = df_media_receita.toPandas()
df_media_receita_pd.to_csv("media_receita_editor.csv", index=False)

+--------------------+----------------------------+
|          Publisher |Média de receita para editor|
+--------------------+----------------------------+
| Hachette Book Group|                     2297.22|
|Penguin Group (US...|                     2033.01|
|HarperCollins Pub...|                     1927.69|
|    Random House LLC|                     1543.93|
|Simon and Schuste...|                      846.63|
|           Macmillan|                      796.92|
|HarperCollins Pub...|                       707.7|
|HarperCollins Chr...|                      533.92|
|Amazon Digital Se...|                      249.59|
+--------------------+----------------------------+



In [ ]:
#23 - Livro que obteve a maior receita para o editor

df_livro_maior_receita = df.groupBy("Book Name", "Author", "Publisher ").agg(sum("publisher revenue").alias("Total de receitas"))
livro_maior_receita = df_livro_maior_receita.orderBy((col("Total de receitas").desc())).first()
print(f"O livro com maior receita para o editor é {livro_maior_receita['Book Name']} publicado pela autora {livro_maior_receita['Author']} na editora {livro_maior_receita['Publisher ']}, com uma receita de {livro_maior_receita['Total de receitas']}.")
df_livro_maior_receita_pd = df_livro_maior_receita.toPandas() #Converte para pandas
df_livro_maior_receita_pd.to_csv("livros_receita_editor.csv", index=False) #Salva em csv

O livro com maior receita para o editor é Go Set a Watchman publicado pela autora Harper Lee na editora Amazon Digital Services,  Inc., com uma receita de 28677.0.
